# A Boolean Retrieval System

In [1]:
from functools import total_ordering, reduce  # not essential but reduces the code we have to write
import csv     # for csv files
import re      # for regular expressions
import pickle  # to save the index
import time
import os.path
import copy

## Postings

A `Posting` object is simply the docID of a document. It has a method `get_from_corpus` that given the corpus retrieves the document corresponding to that docID. Then it has some comparison methods to check if two docID are equal, one greater than the other, etc.

In [2]:
@total_ordering   # takes a class where we have defined at least the methods `eq` and `gt`/`lt` and defines in a consistent way all the other methods (otherwise we should implement them all by hand)
class Posting:

    _docID: int
    _positions: list
    
    def __init__(self, docID: int, pos: list = None):
        """ Class constructor.
        Fields:
          _docID     -- ID of the document in which the term is contained
          _positions -- list of the positions in the document in which the term is present
        """
        self._docID = docID
        if pos != None:
            self._positions = [pos]
        else:
            self._positions = None # sometimes I don't need the list of positions (e.g. in the plist of all the DocID)

    def add(self, pos: int):
        self._positions.append(pos)
    
    def get_from_corpus(self, corpus):  # return from the corpus the doc corresponding to that docID. In the list you only save the docID, not the all document
        """ Returns the document corresponding to that docID from the corpus.
        """
        return corpus[self._docID]
    
    def __eq__(self, other: 'Posting') -> bool:  # euqality comparator
        """ Performs the comparison between this posting and another one.
        Since the ordering of the postings is only given by their docID,
        they are equal when their docIDs are equal.
        """
        return self._docID == other._docID
    
    def __gt__(self, other: 'Posting') -> bool:  # greather than comparator
        """ As in the case of __eq__, the ordering of postings is given
        by the ordering of their docIDs.
        """
        return self._docID > other._docID

    def __hash__(self):
        return hash(repr(self))
    
    def __repr__(self) -> str:
        """ String representation of the class.
        """
        return str(self._docID) + ", pos:" + str(self._positions)

## Posting Lists

A `PostingList` object is a list of `Posting`s. You can construct an empty `PostingList` with `__init__`, or construct and initialize a `PostingList` directly with one docID with `from_docID`, or you can create a `PostingList` object with an already existing list using `from_posting_list`. Then you can merge two posting list with `merge` (the one in input will be added at the end of the one on which the mehod `merge` is called, without any checking on the total ordering of the list), you can intersect them with `intersection` or you can unify them with `union`. With `get_from_corpus` we can retrieve the documents corresponding to the docID stored in this `PostingList`.

In [3]:
class PostingList:

    _postings: list
    
    def __init__(self):
        """ Class constructor.
        """
        self._postings = []    # it has as an attribute a list of posting
        
    @classmethod     # to define another constructor. It will return another PostingList like a constructor
    def from_docID(cls, docID, pos = None) -> 'PostingList':
        """ A posting list can be constructed starting from a single docID.
        """
        plist = cls()
        if pos != None:
            plist._postings = [Posting(docID, pos)]
        else:
            plist._postings = [Posting(docID)]
        return plist
    
    @classmethod
    def from_posting_list(cls, postingList: list) -> 'PostingList':
        """ A posting list can also be constructed by using another posting list.
        """
        plist = cls()
        plist._postings = postingList   # we use it as the postins of this PostingList
        return plist
    
    def merge(self, other: 'PostingList'):  # we have to merge postinglists
        """ Merges the other posting list to this one in a desctructive
        way, i.e., modifying the current posting list. This method assumes
        that all the docIDs of the second list are higher than the ones
        in this list. It assumes the two posting lists to be ordered
        and non-empty. Under those assumptions duplicate docIDs are
        discarded.
        """
        i = 0
        last = self._postings[-1]   # the last element of the current postinglist
        while (i < len(other._postings) and last == other._postings[i]):  # we can have the same docID multiple times (the term is present multiple times in the document) and when we merge them we don't want it multiple times
            if last._positions:
                last._positions.extend(other._postings[i]._positions)
            i += 1
        self._postings += other._postings[i:]
        
    def intersection(self, other: 'PostingList'):
        """ Returns a new posting list resulting from the intersection
        of this one and the one passed as argument.
        """
        intersection = []
        i = 0; j = 0
        while (i < len(self._postings) and j < len(other._postings)):  # until we reach the end of a posting list
            if (self._postings[i] == other._postings[j]):
                intersection.append(self._postings[i])
                i += 1;  j += 1
            elif (self._postings[i] < other._postings[j]):
                i += 1
            else:
                j += 1
        return PostingList.from_posting_list(intersection)
    
    def union(self, other: 'PostingList'):
        """ Returns a new posting list resulting from the union of this
        one and the one passed as argument.
        """
        union = []
        i = 0; j = 0
        while (i < len(self._postings) and j < len(other._postings)):
            if (self._postings[i] == other._postings[j]):
                union.append(self._postings[i])
                i += 1;  j += 1
            elif (self._postings[i] < other._postings[j]):
                union.append(self._postings[i])   # because i is the smallest one
                i += 1
            else:
                union.append(other._postings[j]) 
                j += 1
        for k in range(i, len(self._postings)):  # we have to append the remaining elements of the non emptied list
            union.append(self._postings[k])
        for k in range(j, len(other._postings)):
            union.append(other._postings[k])
        return PostingList.from_posting_list(union)

    def positional_search(self, other: 'PostingList', step: int = 1):
        """ Returns a new posting list resulting from the intersection
        of this one and the one passed as argument.
        """
        match = []
        i = j = 0
        while (i < len(self._postings) and j < len(other._postings)):  # until we reach the end of a posting list
            if (self._postings[i] == other._postings[j]):
                n = m = u = 0
                while (n < len(self._postings[i]._positions) and m < len(other._postings[j]._positions)):
                    if (self._postings[i]._positions[n] + step == other._postings[j]._positions[m]):
                        if u == 0:
                            match.append(Posting(self._postings[i]._docID, self._postings[i]._positions[n]))
                            u = 1
                        else:
                            match[-1].add(self._postings[i]._positions[n])
                        n += 1;  m += 1
                    elif (self._postings[i]._positions[n] < other._postings[j]._positions[m]):
                        n += 1
                    else:
                        m += 1
                i += 1;  j += 1
            elif (self._postings[i] < other._postings[j]):
                i += 1
            else:
                j += 1
        return PostingList.from_posting_list(match)
    
    def get_from_corpus(self, corpus):   # used when we have a posting list that is the result of a query, but I don't want the docID, I want the docs!
        return list(map(lambda x: x.get_from_corpus(corpus), self._postings))  # I return a list of documents
    
    def __getitem__(self, key):
        return self._postings[key]
    
    def __len__(self):
        return len(self._postings)
    
    def __repr__(self):
        return "\n".join(map(str, self._postings))

## Terms

A `Term` object contains both the word itself and the `PostingList` with all the docIDs of the documents in which the word is contained. The `merge` function merges the `PostingList`s of two equal `Term`s. Then we have some comparison methods to check if two `Term`s are equal or one is greater then the other, etc.

In [4]:
class ImpossibleMergeError(Exception):
    pass

In [5]:
@total_ordering  # to have all the ordering methods defined automatically
class Term:

    posting_list: PostingList
    
    def __init__(self, term, docID, pos):   # we create a term with a DocID, we sort them and we merge the equal terms
        self.term = term
        self.posting_list = PostingList.from_docID(docID, pos)
        
    def merge(self, other: 'Term'):   # when we merge two terms
        """ Merges (destructively) this term and the corresponding posting list
        with another equal term and its corrsponding posting list.
        """
        if (self.term == other.term): # cannot merge posting lists with different terms!
            self.posting_list.merge(other.posting_list)  # merge the current posting list with the one of the other
        else: 
            raise ImpossibleMergeError # (some kind of error) error of impossible merge
            
    def __eq__(self, other: 'Term'):
        return self.term == other.term
    
    def __gt__(self, other: 'Term'):
        return self.term > other.term
    
    def __repr__(self):
        return self.term + ": " + repr(self.posting_list)

## Inverted Index

In [6]:
# We have to do some step of tokenization and normalization

def normalize(text):
    """ A simple funzion to normalize a text.
    It removes everything that is not a word, a space or an hyphen and downcases all the text.
    """
    no_punctuation = re.sub(r'[^\w^\s^-]', '', text)  # the text that matches a certain pattern will be substittuted with the second expression. ^\w → not something alphanumeric, ^\s → not some space, ^- → not a dash, replace it with '', the empty string
    downcase = no_punctuation.lower()  # put everything to lower case
    return downcase

def tokenize(movie: 'MovieDescription') -> list:
    """ Returns a list, which is a posting list, from a movie
    description of all tokens present in the description.
    """
    text = normalize(movie.description)
    return list(text.split())

Function to print a progress bar, taken from [here](https://stackoverflow.com/questions/3160699/python-progress-bar).

In [7]:
import time, sys

def update_progress(progress):
    """ Displays or updates a console progress bar.
    Accepts a float between 0 and 1. Any int will be converted to a float.
    A value under 0 represents a 'halt'.
    A value at 1 or bigger represents 100%
    """
    barLength = 40 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done!\r\n"
    block = int(round(barLength*progress))
    text = "\r[{0}] {1}% {2}".format( "#"*block + "."*(barLength-block), round(progress*100, 2), status)
    sys.stdout.write(text)
    sys.stdout.flush()

In an inverted index we store, for each term, the list of documents containing it.

So an `InvertedIndex` object contains a dictionary `_dictionary` with as keys the words themselves and as values the `Term` associated to each word, which, we recall, contains the `PostingList` associated to the word.\
It also stores a list with all the Postings, `complete_plist`, used to answer the NOT queries.

In [8]:
from typing import List

class InvertedIndex:

    _dictionary: List
    complete_plist: PostingList
    
    def __init__(self):
        ''' Empty class constructor.
        Fields:
          _dictionary    -- the collection of all terms that we have in the inverted index
          complete_plist -- PostingList containing all the documents of the corpus
        '''
        self._dictionary = []
        self.complete_plist = PostingList()
        
    @classmethod  # to have multiple constructors. It's like a static method in Java: you call InvertedIndex.from_corpus()
    def from_corpus(cls, corpus: list):
        ''' Class constructor processing a corpus. This is the constructor that creates the index from scatch given a corpus of documents.
        '''
        # Here we "cheat" by using python dictionaries (we should create a big list, sort it and merge everything)
        intermediate_dict = {}
        print("Processing the corpus to create the index...")
        for docID, document in enumerate(corpus): # NB: corpus → collection (list) of objects of type MovieDescription
            # Update complete_plist
            if docID == 0:
                compl_plist = PostingList.from_docID(docID)
            else:
                compl_plist.merge(PostingList.from_docID(docID))
            # Update _dictionary
            tokens = tokenize(document) # document is a MovieDescription object
            for pos, token in enumerate(tokens):
                term = Term(token, docID, pos)
                try:
                    intermediate_dict[token].merge(term)  # I merge the two posting lists → Term.merge() which calls PostingList.merge()
                except KeyError:
                    intermediate_dict[token] = term  # when the term is not present in the dictionary
            # To observe the progressing of our indexing
            update_progress(docID/len(corpus))
        
        idx = cls()  # we call the constructor of the class InvertedIndex
        idx._dictionary = sorted(intermediate_dict.values())  # list of all the sorted terms
        idx.complete_plist = compl_plist
        return idx
    
    def __getitem__(self, key):
        ''' Index the inverted index using as keys the Terms.
        '''
        for term in self._dictionary:  # we could do a binary search
            if term.term == key:
                return term.posting_list  # quering the index with a  word returns the PostingList associated to that word
        raise KeyError(f"The term '{key}' is not present in the index.") # the key is not present!
        
    def __repr__(self):
        ''' Representation of the index.
        '''
        return "A dictionary with " + str(len(self._dictionary)) + " terms"

## Reading the Corpus

A `MovieDescription` object has a title and a description.  We have some comparison methods to check if two `MovieDescription`s are equal or one is greater then the other, etc. The function `hash` computes the hash of a `MovieDescription` using the hash of its title and its description.

We have implemented the comparison methods to make `MovieDescription` a sortable object (so we can iterate on it), and the `hash` method to make it hashable (so we can put it in a `set`).

In [9]:
@total_ordering
class MovieDescription:  # container for all the info we have about the movie
    
    def __init__(self, title: str, description: str):
        self.title = title
        self.description = description
        
    def __eq__(self, other: 'MovieDescription'):
        return self.title == other.title
    
    def __gt__(self, other: 'MovieDescription'):
        return self.title > other.title

    def __hash__(self):
      return hash((self.title, self.description))
        
    def __repr__(self):
        return self.title  # + "\n" + self.description + "\n"

In [10]:
def read_movie_descriptions():
    filename = 'data/plot_summaries.txt'   # not very portable but done for the sake of simplicity
    movie_names_file = 'data/movie.metadata.tsv'
    with open(movie_names_file, 'r') as csv_file:
        movie_names = csv.reader(csv_file, delimiter = '\t')   # we define the csv reader
        names_table = {}   # Python dictionary with all the names of the films: key = movieID, value = movie title
        for name in movie_names:
            names_table[name[0]] = name[2] # the first element is the ID, the third elemnt is the title
    # Now we have all the associations between ID and title, we miss the move description

    with open(filename, 'r') as csv_file:
        descriptions = csv.reader(csv_file, delimiter = '\t')
        corpus = []   # collection (list) of objects of type MovieDescription
        for desc in descriptions:
            try:      # at least in this dataset there are some errors so some descriptions have not a matching ID
                movie = MovieDescription(names_table[desc[0]], desc[1]) # the first element is the ID, the second the description
                corpus.append(movie)
            except KeyError:  # in case we don't find the title associated to that ID
                # We ignore the descriptions for which we cannot find a title
                pass
        return corpus

## Edit distance

By computing the edit distance we can find the set of words that are the closest to a misspelled word. However, computing the edit distance on the entire dictionary can be too expensive. We can use some heuristics to limit the number of words, like looking only at words with the same initial letter (hopefully this has not been misspelled). The latter is what is implemented in this model.

In [11]:
def edit_distance(u, v, print = False):
    """ Computes the edit (or Levenshtein) distance between two words u and v.
    """
    nrows = len(u) + 1
    ncols = len(v) + 1
    M = [[0] * ncols for i in range(0, nrows)]  # matrix all filled with zeros
    for i in range(0, nrows):  # we fill the first row, the trivial one
        M[i][0] = i
    for j in range(0, ncols):  # we fill the first col, the trivial one
        M[0][j] = j
    for i in range(1, nrows):
        for j in range(1, ncols):
            candidates = [M[i-1][j] + 1, M[i][j-1] + 1]
            if (u[i-1] == v[j-1]):
                candidates.append(M[i-1][j-1])
            else:
                candidates.append(M[i-1][j-1] + 1)
            M[i][j] = min(candidates)
            # To print the distance matrix
            if print:
                print(M[i][j], end="\t")
        if print:
          print()
    return M[-1][-1]  # Bottom right element of M (-1 means the last element)

def find_nearest(word, dictionary, keep_first=False):
    if keep_first:
        # If keep_first is true then we only search across the words in the dictionary starting with the same letter
        dictionary = [w for w in dictionary if w[0] == word[0]]
    # Remove comment to see the reduction in the size of the dictionary when keeping fixed the first letter
    #print(len(dictionary))
    # Apply f(x) = edit_distance(word, x) to all words in the dictionary
    distances = map(lambda x: edit_distance(word, x), dictionary)
    # Produce all the pairs (distance, term) usng zip and find one with the minimal distance.
    return min(zip(distances, dictionary))[1]

## IR System

An `IRsystem` object contains the entire corpus and the `InvertedIndex`.

In [12]:
class IRsystem:

    _corpus: list
    _index: InvertedIndex
    
    def __init__(self, corpus: list, index: 'InvertedIndex'):
        self._corpus = corpus
        self._index = index
        
    @classmethod
    def from_corpus(cls, corpus: list): # generate the entire inverted index calling the constructor
        index = InvertedIndex.from_corpus(corpus)
        return cls(corpus, index)  # retrun the constructor when we have yet the index

    def get_from_corpus(self, plist):
        return plist.get_from_corpus(self._corpus)

    def spelling_correction(self, norm_words: List[str]):
        ''' Performs a spelling correction of the normalized words finding the nearest words.
        '''
        postings = []
        for w in norm_words:
            try:
                res = self._index[w]
            except KeyError:
                dictionary = [t.term for t in self._index._dictionary]
                sub = find_nearest(w, dictionary, keep_first=True)
                print("{} not found. Did you mean {}?".format(w, sub))
                res = self._index[sub]
            postings.append(res)
        return postings

    def answer_and_query(self, words: List[str], spellingCorrection = False):
        """ AND-query
        Arguments:
          spellingCorrection -- if `True` a spelling correction is performed
        """
        norm_words = map(normalize, words)  # Normalize all the words. IMPORTANT!!! If the user uses upper-case we will not have ANY match! We have to perform the same normalization of the docs in the corpus on the query!
        if not spellingCorrection:
            postings = map(lambda w: self._index[w], norm_words) # get the posting list for each word → list of posting lists
        else:
            postings = self.spelling_correction(norm_words)
        plist = reduce(lambda x, y: x.intersection(y), postings)  # apply the function to the two items of the list, then apply it to the result with the third, then the result with the fourt term and so on until the end of the list
        return self.get_from_corpus(plist)

    def answer_or_query(self, words: List[str], spellingCorrection = False):
        """ OR-query
        Arguments:
          spellingCorrection -- if `True` a spelling correction is performed
        """
        norm_words = map(normalize, words)
        if not spellingCorrection:
            postings = map(lambda w: self._index[w], norm_words)
        else:
            postings = self.spelling_correction(norm_words)
        plist = reduce(lambda x, y: x.union(y), postings)
        return self.get_from_corpus(plist)

    def answer_not_query(self, words: List[str], spellingCorrection = False):
        """ NOT-query (if `words` is longer than 1, the words are connected using an AND and then the NOT is performed)
        Arguments:
          spellingCorrection -- if `True` a spelling correction is performed
        """
        norm_words = map(normalize, words)
        if not spellingCorrection:
            postings = map(lambda w: self._index[w], norm_words)
        else:
            postings = self.spelling_correction(norm_words)
        words_plist = reduce(lambda x, y: x.union(y), postings)
        plist = copy.deepcopy(self._index.complete_plist)
        for i in words_plist:
            if i in plist:
                plist._postings.remove(i)
        return self.get_from_corpus(plist)

    def answer_query(self, op: str, words = None, word = None, postings = None, postings2 = None, NOT_switch = False, spellingCorrection = False):
        ''' Complex query.
        Arguments:
          NOT_switch -- Used to switch the order of the two posting lists `postings` and `postings2` in the NOT query, since this operator is not commutative.
          spellingCorrection -- if `True` a spelling correction is performed
        '''
        if words:
            norm_word_0 = normalize(words[0])
            norm_word_1 = normalize(words[1])
            if not spellingCorrection:
                postings = self._index[norm_word_0]
                postings2 = self._index[norm_word_1]
            else:
                postings = self.spelling_correction([norm_word_0])[0]
                postings2 = self.spelling_correction([norm_word_1])[0]
        elif word:
            norm_word = normalize(word)
            if not spellingCorrection:
                postings2 = self._index[norm_word]
            else:
                postings2 = self.spelling_correction([norm_word])[0]

        if op == 'AND':
            plist = postings.intersection(postings2)
            return plist
        elif op == 'OR':
            plist = postings.union(postings2)
            return plist
        elif op == 'NOT':
            if not NOT_switch: # used to control the order of postings and postings2
                postings_copy = copy.deepcopy(postings)
                for i in postings2:
                    if i in postings_copy:
                        postings_copy._postings.remove(i)
                return postings_copy
            else:
                postings2_copy = copy.deepcopy(postings2)
                for i in postings:
                    if i in postings2_copy:
                        postings2_copy._postings.remove(i)
                return postings2_copy

    def answer_phrase_query(self, words: List[str], spellingCorrection = False):
        """ Phrase-query
        Arguments:
          spellingCorrection -- if `True` a spelling correction is performed
        """
        norm_words = map(normalize, words)  # Normalize all the words. IMPORTANT!!! If the user uses upper-case we will not have ANY match! We have to perform the same normalization of the docs in the corpus on the query!
        if not spellingCorrection:
            postings = map(lambda w: self._index[w], norm_words) # get the posting list for each word → list of posting lists
        else:
            postings = self.spelling_correction(norm_words)
        # Reduce
        it = enumerate(iter(postings))
        i, plist = next(it)
        for i, element in it:
            # i is the step 
            plist = plist.positional_search(element, i)
        return plist, self.get_from_corpus(plist)

    def answer_phrase_query_ksteps(self, term1: str, term2: str, k: int, spellingCorrection = False):
        """ Phrase-query of the form 'term1 /k term2'
        Arguments:
          spellingCorrection -- if `True` a spelling correction is performed
        """
        norm_term1 = normalize(term1)
        norm_term2 = normalize(term2)
        if not spellingCorrection:
            postings1 = self._index[norm_term1]
            postings2 = self._index[norm_term2]
        else:
            postings1 = self.spelling_correction(norm_term1)
            postings2 = self.spelling_correction(norm_term2)
        plist = []
        for i in range(1, k+2): # k+1 because I have to count also the last word, and +1 again because otherwise 'range' is [1,k+1) and I need [1, k+1]
            plist.append(postings1.positional_search(postings2, i))
        plist = reduce(lambda x, y: x.union(y), plist)
        return plist, self.get_from_corpus(plist)

    def answer_trailing_wildcard(self, wildcard):
        plist = []
        for w in self._index._dictionary:
            if starts_with(w.term, wildcard):
                plist.append(w.posting_list)
        plist = reduce(lambda x, y: x.union(y), plist)
        return self.get_from_corpus(plist)

In [13]:
def starts_with(word, start):
    n = len(start)
    if word[:n] == start:
        return True
    else:
        return False

## Queries

In [14]:
def print_result(answer: PostingList, spellingCorrection=False):
    if spellingCorrection: # for a better output
        print()
    for movie in answer:
        print(movie)

In [15]:
def and_query(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    words = text.split()
    answer = ir.answer_and_query(words, spellingCorrection)  # list of documents
    if not noprint:
        print_result(answer, spellingCorrection)
    return answer
        
def or_query(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    words = text.split()
    answer = ir.answer_or_query(words, spellingCorrection)
    if not noprint:
        print_result(answer, spellingCorrection)
    return answer

def not_query(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    words = text.split()
    answer = ir.answer_not_query(words, spellingCorrection)
    if not noprint:
        print_result(answer, spellingCorrection)
    return answer

def query(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    """ This function can answer complex queries with 'AND', 'OR' and 'NOT' but without parentheses.
    E.g. text = "yoda AND darth OR Gandalf NOT love"
    """
    words = text.split()
    if len(words) == 1:
        print("You cannot use one single word! Use at least two words connected with a logical operator.")
        return None
    for i, w in enumerate(words):
        if w in ['AND', 'OR', 'NOT']:
            if i == 1:
                plist = ir.answer_query(op = w, words = [words[i-1], words[i+1]], spellingCorrection=spellingCorrection)
            else:
                plist = ir.answer_query(op = w, word = words[i+1], postings = plist, spellingCorrection=spellingCorrection)
    answer = ir.get_from_corpus(plist)
    if not noprint:
        print_result(answer, spellingCorrection)
    return answer

In [16]:
def query_with_pars(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    """ This function can answer to any type of query, also complex ones. Use 'AND', 'OR' and 'NOT'
    and parenthesis to specify how to combine the words in the query.
    E.g. text = "(yoda AND darth) OR Gandalf NOT love"
    """
    # add a space after '(' and before ')' so to split them into separate tokens
    text = re.sub(r"\(", r'( ', text)
    text = re.sub(r"\)", r' )', text)
    # split the text in words (it eliminates spaces at the beginning/end of words, so even if we add them with the first passage it's not important)
    words = text.split()

    if len(words) == 1 or (len(words) == 3 and ("(" in words or ")" in words)):
        print("You cannot use one single word! Use at least two words connected with a logical operator.")
        return None

    # words_mod = copy.deepcopy(words) # For now not necessary, I don't reuse the array 'words'
    words_mod = words                  # I write it like this so it is easy to change is 'words' will be needed
    # Wrap the text in two parentheses so to perform the following while one last time, avoiding to repeat code, and compute the whole query
    words_mod.insert(0, "(")
    words_mod.append(")")
    
    openp = []
    closep = []
    for i, w in enumerate(words_mod):
        if w == "(": openp.append(i)
        elif w == ")": closep.append(i)
    
    if len(openp) != len(closep):
        print("The number of open parentheses is different from the number of closed parentheses.")
        return None

    while closep:
        c = closep[0]
        o = None
        for i in openp:
            if i < c:
                o = i
        #plist = PostingList()  # wrong, since if there are extra parentheses around all the query I overwrite the last plist and I store in words_mod an empty plist
        # Process the query in these parenthesis, the remove o in openp and c in closep
        for i, w in enumerate(words_mod[o+1 : c]):
            if w in ['AND', 'OR', 'NOT']:
                if i == 1:
                    item1 = words_mod[(o+1) + i-1]
                    item2 = words_mod[(o+1) + i+1]
                    # Checks to call `ir.answer_query` in the right way
                    if type(item1) == str and type(item2) == str:
                        plist = ir.answer_query(op = w, words = [item1, item2], spellingCorrection=spellingCorrection)
                    elif type(item1) == PostingList and type(item2) == str:
                        plist = ir.answer_query(op = w, word = item2, postings = item1, spellingCorrection=spellingCorrection)
                    elif type(item1) == str and type(item2) == PostingList:
                        plist = ir.answer_query(op = w, word = item1, postings = item2, NOT_switch = True, spellingCorrection = spellingCorrection)
                    elif type(item1) == PostingList and type(item2) == PostingList:
                        plist = ir.answer_query(op = w, postings = item1, postings2 = item2, spellingCorrection=spellingCorrection)
                else:
                    item2 = words_mod[(o+1) + i+1]
                    if type(item2) == str:
                        plist = ir.answer_query(op = w, word = item2, postings = plist, spellingCorrection=spellingCorrection)
                    elif type(item2) == PostingList:
                        plist = ir.answer_query(op = w, postings = plist, postings2 = item2, spellingCorrection=spellingCorrection)
        words_mod[o] = plist
        del words_mod[o+1:c+1]
        openp = []
        closep = []
        for i, w in enumerate(words_mod):
            if w == "(": openp.append(i)
            elif w == ")": closep.append(i)
    
    answer = ir.get_from_corpus(plist)
    if not noprint:

        print_result(answer, spellingCorrection)
    return answer

## Phrase queries

### Answering a phrase query with positional indexing

One way to answer a phrase query is to add, for each posting, the set of positions in which the term appear in the document. We perform something like the intersection only that now we have to go inside and check if the two terms appear in adjacent positions. So we search if they are contained in the same document and if they are one after the other.

Besides, the positional index can be used to support the operators of the form "$\texttt{term}_1 /k \texttt{ term}_2$" with $k$ an integer indicating the maximum number of words that can be between $\texttt{term}_1$ and $\texttt{term}_2$.

In [17]:
def phrase_query(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    ''' This function can answer to a phrase query of any length.
    '''
    words = text.split()
    posting_list, answer = ir.answer_phrase_query(words, spellingCorrection)
    if not noprint:
        print_result(answer, spellingCorrection)
    return posting_list, answer

In [18]:
def phrase_query_ksteps(ir: IRsystem, text: str, spellingCorrection=False, noprint=True):
    ''' This function can answer a phrase query of the form 'term1 /k term2'  with k an integer
    indicating the maximum number of words that can be between term1 and term2.
    '''
    words = text.split()
    if len(words) != 3:
        print("ERROR: for now you can only use a query of the form: 'term1 /k term2' with k an integer.")
    if words[1][0] == '/':
        k = int(words[1][1:])
    else:
        print("ERROR: for now you can only use a query of the form: 'term1 /k term2' with k an integer.")
    posting_list, answer = ir.answer_phrase_query_ksteps(words[0], words[2], k, spellingCorrection)
    if not noprint:
        print_result(answer, spellingCorrection)
    return posting_list, answer

## Wildcard queries

Ho anche una domanda per quanto riguarda l'implementazione. A lezione abbiamo visto, per le wildcard queries, che se si usano gli alberi binari una trailing wildcard "ca*" consiste nel un sottoalbero del termine "ca", purtroppo però non sono riuscita a trovare una libreria di Python per gli alberi bilanciati (ho guardato anche per BTrees) che autobilanci l'albero da sola ma che mi faccia anche accedere ai nodi per poter selezionare il sottoalbero che mi interessa. Stavo dunque pensando di continuare ad usare un Python dictionary per l'indice e creare una lista di tutti i termini che iniziano con la wildcard "ca", cercarli nel dizionario e poi unirli. Come implementazione delle wildcard queries andrebbe bene lo stesso?

### Trailing wildcards

In a **trailing wildcard** there is only one wildcard and it is at the end of the word, like **term\***, in which we want everything starting with "term".

To answer the query we can retrieve the posting lists of all the terms that starts with "term" and then perform a union of the results.

In [19]:
def trailing_wildcard(ir: IRsystem, text: str, noprint=True):
    wildcard = text.split('*')[0]
    answer = ir.answer_trailing_wildcard(wildcard)  # list of documents
    if not noprint:
        print_result(answer)
    return answer

In [20]:
txt = "Hello World"[::-1]
print(txt)

print()
txt = "Hello World$"

def rotations(txt):
    for i in range(len(txt)):
        first = txt[0:i]
        second = txt[i:]
        print(second + first)
        
rotations(txt)

dlroW olleH

Hello World$
ello World$H
llo World$He
lo World$Hel
o World$Hell
 World$Hello
World$Hello 
orld$Hello W
rld$Hello Wo
ld$Hello Wor
d$Hello Worl
$Hello World


## Test queries

### Initialization

In [21]:
corpus = read_movie_descriptions()
len(corpus)

42204

#### Saving / loading the index

We will save the index using `Pickle`. `Pickle` is used for serializing and de-serializing Python object structures, also called marshalling or flattening. Serialization refers to the process of converting an object in memory to a byte stream that can be stored on disk or sent over a network. Later on, this character stream can then be retrieved and de-serialized back to a Python object.

In [22]:
updated = True

filename = "index.pickle"

# If the index is saved and it is updated I load it, otherwise I create it and save it
if os.path.isfile(filename) and updated:
    print ("Index file exists. Loading the index...")
    # load the index
    tic = time.time()
    infile = open(filename, 'rb')
    idx = pickle.load(infile)
    infile.close()
    toc = time.time()
    print("Index loaded.")
    print(f"Time: {round(toc-tic, 3)}s")
else:
    print ("Index file does not exist.")
    tic = time.time()
    idx = InvertedIndex.from_corpus(corpus)
    toc = time.time()
    print(f"\n\nTime: {round(toc-tic, 3)}s")
    # save the index
    outfile = open(filename, 'wb')
    pickle.dump(idx, outfile)
    outfile.close()

Index file exists. Loading the index...
Index loaded.
Time: 26.818s


In [23]:
print(idx)

A dictionary with 194757 terms


In [24]:
ir = IRsystem(corpus, idx)

In [25]:
try:
  ir.get_from_corpus(ir._index[normalize("thig")])
except KeyError:
    print(sys.exc_info()[1])

"The term 'thig' is not present in the index."


### AND queries

In [26]:
fg_and_query = and_query(ir, "frodo Gandalf", noprint=False)

The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
The Hunt for Gollum
The Return of the King
Date Movie
The Lord of the Rings: The Two Towers
The Lord of the Rings: The Return of the King


In [27]:
yld_and_query = and_query(ir, "yoda Luke darth", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Something, Something, Something Dark Side
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
It's a Trap!


In [28]:
frodo_query = ir.get_from_corpus(ir._index[normalize("frodo")])
frodo_set = set(frodo_query)

gandalf_query = ir.get_from_corpus(ir._index[normalize("Gandalf")])
gandalf_set = set(gandalf_query)

fg_and_set = frodo_set.intersection(gandalf_set)

assert set(fg_and_query) == fg_and_set

In [29]:
yoda_query = ir.get_from_corpus(ir._index[normalize("yoda")])
yoda_set = set(yoda_query)

luke_query = ir.get_from_corpus(ir._index[normalize("Luke")])
luke_set = set(luke_query)

darth_query = ir.get_from_corpus(ir._index[normalize("darth")])
darth_set = set(darth_query)

yld_and_set = yoda_set.intersection(luke_set).intersection(darth_set)

assert set(yld_and_query) == yld_and_set

#### With spelling correction

In [30]:
mispelled_and_query = and_query(ir, "yioda lukke darhth", spellingCorrection=True, noprint=False)

yioda not found. Did you mean yoda?
lukke not found. Did you mean luke?
darhth not found. Did you mean darth?

Star Wars Episode V: The Empire Strikes Back
Something, Something, Something Dark Side
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
It's a Trap!


In [31]:
assert yld_and_query == mispelled_and_query

### OR queries

In [32]:
fy_or_query = or_query(ir, "frodo yoda", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Star Wars Episode II: Attack of the Clones
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
Something, Something, Something Dark Side
The Hunt for Gollum
The Return of the King
Return of the Ewok
Aliens in the Wild, Wild West
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Star Wars: The Clone Wars
Date Movie
Gulliver's Travels
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [33]:
fy_or_set = set(frodo_query + yoda_query)

assert set(fy_or_query) == fy_or_set

In [34]:
fyg_or_query = or_query(ir, "frodo yoda gandalf", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Imaginationland Episode II
Star Wars Episode II: Attack of the Clones
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
Something, Something, Something Dark Side
The Hunt for Gollum
The Return of the King
Return of the Ewok
Aliens in the Wild, Wild West
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Star Wars: The Clone Wars
Date Movie
Gulliver's Travels
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [35]:
fyg_or_set = set(frodo_query + yoda_query + gandalf_query)

assert set(fyg_or_query) == fyg_or_set

In [36]:
love_query = ir.get_from_corpus(ir._index[normalize("love")])
fyl_or_query = or_query(ir, "frodo yoda love")
fyl_or_set = set(frodo_query + yoda_query + love_query)

assert set(fyl_or_query) == fyl_or_set

#### With spelling correction

In [37]:
mispelled_or_query = or_query(ir, "frodoo yioda ganalf", spellingCorrection=True, noprint=False)

frodoo not found. Did you mean frodo?
yioda not found. Did you mean yoda?
ganalf not found. Did you mean gandalf?

Star Wars Episode V: The Empire Strikes Back
Imaginationland Episode II
Star Wars Episode II: Attack of the Clones
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
Something, Something, Something Dark Side
The Hunt for Gollum
The Return of the King
Return of the Ewok
Aliens in the Wild, Wild West
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Star Wars: The Clone Wars
Date Movie
Gulliver's Travels
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [38]:
assert fyg_or_query == mispelled_or_query

### NOT queries

In [39]:
#a_not_query = not_query(ir, "a", noprint=True)

In [40]:
corpus_set = set(corpus)
a_query = ir.get_from_corpus(ir._index[normalize("a")])
a_set = set(a_query)
a_not_set = corpus_set.difference(a_set)

#assert set(a_not_query) == a_not_set

In [41]:
#lm_not_query = not_query(ir, "love mother", noprint=True)

In [42]:
love_set = set(love_query)
mother_query = ir.get_from_corpus(ir._index[normalize("mother")])
mother_set = set(mother_query)
lm_set = love_set.union(mother_set)
lm_not_set = corpus_set.difference(lm_set)

#assert set(lm_not_query) == lm_not_set

In [43]:
#yg_not_query = not_query(ir, "yoda Gandalf", noprint=True)

In [44]:
yg_set = yoda_set.union(gandalf_set)
yg_not_set = corpus_set.difference(yg_set)

#assert set(yg_not_query) == yg_not_set

#### With spelling correction

In [45]:
#mispelled_a_not_query = not_query(ir, "aq", spellingCorrection=True, noprint=True)

In [46]:
#assert a_not_query == mispelled_a_not_query

### Compex queries

In [47]:
query(ir, "yoda", noprint=False)

You cannot use one single word! Use at least two words connected with a logical operator.


In [48]:
yAdOg_query = query(ir, "yoda AND darth OR Gandalf", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Imaginationland Episode II
Star Wars Episode II: Attack of the Clones
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
Something, Something, Something Dark Side
The Hunt for Gollum
The Return of the King
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Date Movie
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [49]:
yd_and_set = yoda_set.intersection(darth_set)
yAdOg_set = yd_and_set.union(gandalf_set)

assert set(yAdOg_query) == yAdOg_set

In [50]:
yAdOg_query2 = query_with_pars(ir, "yoda AND darth OR Gandalf", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Imaginationland Episode II
Star Wars Episode II: Attack of the Clones
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
Something, Something, Something Dark Side
The Hunt for Gollum
The Return of the King
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Date Movie
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [51]:
assert set(yAdOg_query2) == yAdOg_set

In [52]:
yOdAg_query = query(ir, "yoda OR darth AND Gandalf", noprint=False)

In [53]:
yd_or_set = yoda_set.union(darth_set)
yOdAg_set = yd_or_set.intersection(gandalf_set)

assert set(yOdAg_query) == yOdAg_set

In [54]:
yOdOgAl_query = query(ir, "yoda OR darth OR Gandalf AND love", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Star Wars Episode II: Attack of the Clones
Wishology
Something, Something, Something Dark Side
Date Movie
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
The Lord of the Rings: The Return of the King


In [55]:
ydg_and_set = yoda_set.union(darth_set).union(gandalf_set)
yOdOgAl_set = ydg_and_set.intersection(love_set)

assert set(yOdOgAl_query) == yOdOgAl_set

In [56]:
yAdOgNl_query = query(ir, "yoda AND darth OR Gandalf NOT love", noprint=False)

Imaginationland Episode II
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
The Hunt for Gollum
The Return of the King
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
It's a Trap!
LEGO Star Wars: Revenge of the Brick


In [57]:
yAdOgNl_set = yAdOg_set.difference(love_set)

assert set(yAdOgNl_query) == yAdOgNl_set

In [58]:
yNdOg_query = query(ir, "yoda NOT darth OR Gandalf", noprint=False)

Imaginationland Episode II
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
The Hunt for Gollum
The Return of the King
Aliens in the Wild, Wild West
Star Wars: The Clone Wars
Date Movie
Gulliver's Travels
The Lord of the Rings: The Two Towers
The Lord of the Rings: The Return of the King


In [59]:
yNd_set = yoda_set.difference(darth_set)
yNdOg_set = yNd_set.union(gandalf_set)

assert set(yNdOg_query) == yNdOg_set

#### Using parentheses

In [60]:
query_with_pars(ir, "(yoda)", noprint=False)

You cannot use one single word! Use at least two words connected with a logical operator.


In [61]:
pyAdpOgNl_query = query_with_pars(ir, "(yoda AND darth) OR Gandalf NOT love", noprint=False)

Imaginationland Episode II
George Lucas in Love
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
The Hunt for Gollum
The Return of the King
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
It's a Trap!
LEGO Star Wars: Revenge of the Brick


In [62]:
assert set(pyAdpOgNl_query) == yAdOgNl_set

In [63]:
yApdOgpNl_query = query_with_pars(ir, "yoda AND (darth OR Gandalf) NOT love", noprint=False)

George Lucas in Love
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
It's a Trap!
LEGO Star Wars: Revenge of the Brick


In [64]:
dOg_set = darth_set.union(gandalf_set)
yApdOgpNl = yoda_set.intersection(dOg_set).difference(love_set)

assert set(yApdOgpNl_query) == yApdOgpNl

In [65]:
yOgApdOlp_complex_query = query_with_pars(ir, "(yoda OR Gandalf AND (darth OR love))", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Star Wars Episode II: Attack of the Clones
George Lucas in Love
Something, Something, Something Dark Side
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Date Movie
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [66]:
dOl_set = darth_set.union(love_set)
yOgApdNlp_set = yoda_set.union(gandalf_set).intersection(dOl_set)

assert set(yOgApdOlp_complex_query) == yOgApdNlp_set

In [67]:
yOpgApdOlpNmpOphNap_complex_query = query_with_pars(ir, "yoda OR (Gandalf AND (darth OR love) NOT mother) OR (hello NOT a)", noprint=False)

Star Wars Episode V: The Empire Strikes Back
Bimbo's Express
Star Wars Episode II: Attack of the Clones
George Lucas in Love
Touchstone: Dancing With Angels
Something, Something, Something Dark Side
Return of the Ewok
Aliens in the Wild, Wild West
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Star Wars: The Clone Wars
Gulliver's Travels
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [68]:
gApdOlpNm_set = gandalf_set.intersection(dOl_set).difference(mother_set)
hello_set = set(ir.get_from_corpus(ir._index[normalize("hello")]))
hNa_set = hello_set.difference(a_set)
yOpgApdOlpNmpOphNap_set = yoda_set.union(gApdOlpNm_set).union(hNa_set)

assert set(yOpgApdOlpNmpOphNap_complex_query) == yOpgApdOlpNmpOphNap_set

In [69]:
test = "hello OR ((how AND (are OR you) OR I AND (am AND fine) OR I) AND am AND (sleepy OR hungry) AND cold)"
hOphApaOypOiApaAfpOiAaApsOhpAcp_query = query_with_pars(ir, test, noprint=False)

Dark Water
House
Pantry Panic
The Jazz Singer
A Witch's Tangled Hare
Look Who's Talking Too
The Life of Reilly
Bimbo's Express
Hansel and Gretel
Phone Booth
Queen of the Damned
Ghost Town
Ghosts
Touchstone: Dancing With Angels
The Weather Man
The Friendly Ghost
Say Hello to Yesterday
Grave Encounters
Duets
The Strangers
Hello Dolly!
Martian Through Georgia
The Daffy Doc
A Star Is Born
Heartlands
Crazy People
WALL-E
A Star Is Born
Sweet Home Alabama
Takeshis'
Closer
Old School
The Nutcracker Prince
Jazzin' for Blue Jean
Prom Night II
The Day After
Bugs and Thugs
I Could Go On Singing
Din of Celestial Birds
A Fine Feathered Frenzy
One False Move
Psychomania
Trapped in the Closet Chapters 13–22
Easy Money
Atlantic Rhapsody
Love Bites
Tom and Cherie
Sleepless in Seattle
Yummy Yummy
The Adventure of Iron Pussy
Tetsuo: The Iron Man
An Affair to Remember
Motel Hell
I'm Here
28 Days Later
Jerry Maguire
The Hole
Love Affair
Basil
The Stepford Wives
Wake Up Jeff
Claire Dolan
The In-Laws


In [70]:
are_set = set(ir.get_from_corpus(ir._index[normalize("are")]))
you_set = set(ir.get_from_corpus(ir._index[normalize("you")]))
aOy_set = are_set.union(you_set)
am_set = set(ir.get_from_corpus(ir._index[normalize("am")]))
aAf_set = am_set.intersection(set(ir.get_from_corpus(ir._index[normalize("fine")])))
i_set = set(ir.get_from_corpus(ir._index[normalize("I")]))
how_set = set(ir.get_from_corpus(ir._index[normalize("how")]))
hApaOypOiApaAfpOi_set = how_set.intersection(aOy_set).union(i_set).intersection(aAf_set).union(i_set)
sleepy_set = set(ir.get_from_corpus(ir._index[normalize("sleepy")]))
hungry_set = set(ir.get_from_corpus(ir._index[normalize("hungry")]))
sOh_set = sleepy_set.union(hungry_set)
cold_set = set(ir.get_from_corpus(ir._index[normalize("cold")]))
hApaOypOiApaAfpOiAaApsOhpAc_set = hApaOypOiApaAfpOi_set.intersection(am_set).intersection(sOh_set).intersection(cold_set)
hOphApaOypOiApaAfpOiAaApsOhpAcp_set = hello_set.union(hApaOypOiApaAfpOiAaApsOhpAc_set)

assert set(hOphApaOypOiApaAfpOiAaApsOhpAcp_query) == hOphApaOypOiApaAfpOiAaApsOhpAcp_set

#### With spelling correction

In [71]:
mispelled_yNdOg_query = query(ir, "yioda NOT ddarth OR Ganalf", spellingCorrection=True, noprint=False)

yioda not found. Did you mean yoda?
ddarth not found. Did you mean darth?
ganalf not found. Did you mean gandalf?

Imaginationland Episode II
The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings
The Hunt for Gollum
The Return of the King
Aliens in the Wild, Wild West
Star Wars: The Clone Wars
Date Movie
Gulliver's Travels
The Lord of the Rings: The Two Towers
The Lord of the Rings: The Return of the King


In [72]:
assert yNdOg_query == mispelled_yNdOg_query

In [73]:
mispelled_yOpgApdOlpNmpOphNap_complex_query = query_with_pars(ir, "yioda OR (Ganalf AND (ddarth OR lovve) NOT motther) OR (helloo NOT aq)", spellingCorrection=True, noprint=False)

ddarth not found. Did you mean darth?
lovve not found. Did you mean love?
ganalf not found. Did you mean gandalf?
motther not found. Did you mean mother?
helloo not found. Did you mean hello?
aq not found. Did you mean a?
yioda not found. Did you mean yoda?

Star Wars Episode V: The Empire Strikes Back
Bimbo's Express
Star Wars Episode II: Attack of the Clones
George Lucas in Love
Touchstone: Dancing With Angels
Something, Something, Something Dark Side
Return of the Ewok
Aliens in the Wild, Wild West
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
Star Wars: The Clone Wars
Gulliver's Travels
Lego Star Wars: The Quest for R2-D2
The Lord of the Rings: The Two Towers
It's a Trap!
The Lord of the Rings: The Return of the King
LEGO Star Wars: Revenge of the Brick


In [74]:
assert yOpgApdOlpNmpOphNap_complex_query == mispelled_yOpgApdOlpNmpOphNap_complex_query

### Phrase queries

In [75]:
text = "Great Britain"
gb_posting_list, gb_phrase_query = phrase_query(ir, text, noprint=False)

Noises Off...
13 Rue Madeleine
The Fox of Glenarvon
International Velvet
Twelve O'Clock High
Saraband for Dead Lovers
Veronico Cruz
Legally Blondes
The Entertainer
The Adams Chronicles
Zulu Dawn
The Three Musketeers
The Story of Adele H.
A Dangerous Man: Lawrence After Arabia
Vigil in the Night
28 Weeks Later
Britannic
The First of the Few
Suez
Hopscotch
It Happened Here
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
The Omen
Patton
Freddie as F.R.O.7
Tristan & Isolde
Garfield: A Tail of Two Kitties
Burn!
Battle of Britain
Tea With Mussolini
The Scarlet Pimpernel
Valiant
The Patriot
Land and Freedom
Sylvia
The Monster X Strikes Back/Attack the G8 Summit
28 Days Later
Jackboots on Whitehall
Omen III: The Final Conflict
Chariots of Fire
John Adams
Souls at Sea
Banned From Television
Satellite in the Sky
The Man Who Shot Liberty Valance
Doomsday
Wild Wild West
633 Squadron
Incense for the Damned


In [76]:
def check_phrase_query(text, posting_list):
    text = normalize(text)
    words = list(text.split())
    for posting in posting_list:
        l = len(words)
        doc = posting._docID
        pos = posting._positions
        tokens = tokenize(corpus[doc])
        for p in pos:
            assert tokens[p:p+l] == words

In [77]:
check_phrase_query(text, gb_posting_list)

In [78]:
text = "New York city"
ny_posting_list, ny_phrase_query = phrase_query(ir, text, noprint=True)

In [79]:
check_phrase_query(text, ny_posting_list)

In [80]:
text = "one of the"
oft_posting_list, oft_phrase_query = phrase_query(ir, text, noprint=False)

Sing
Up and Down
Forest of the Damned 2
Pieces
Geometria
Liar Liar
Day Watch
A Game for Vultures
Suryavamsam
The Policeman
Witness Insecurity
Three Little Bops
Neverland
Maneater
Betty Blue
Phir Kabhi
Two Evil Eyes
Northern Lights
Fantastic Planet
Enemies Among Us
Billa 2
Love, Wedding, Marriage
The Mechanical Monsters
The Dawning
Protector
An Eye for an Eye
Stage Beauty
A Chump at Oxford
Naughty Marietta
Big Nothing
Overnight
Farewell My Concubine
The Andromeda Strain
Apna Sapna Money Money
Ceiling Zero
Sivandha Mann
Winter Cicada
Tales of Manhattan
Batman: Mystery of the Batwoman
Appleseed
Malèna
Jay and Silent Bob Strike Back
StreetDance 3D
China Moon
Warlords of the 21st Century
Scat Cats
The Naked Gun: From the Files of Police Squad!
I Witness
Endangered Species
Super
Armitage III
Dasavatharam
Barbie Mariposa
Feast
Necessary Roughness
No Good Deed
The Last Emperor
Ice Twisters
Chidambaram
The Housemaid
The Cat from Outer Space
Doom
All Quiet on the Western Front
Diljale
Saturday's

In [81]:
check_phrase_query(text, oft_posting_list)

In [82]:
text = "United States of America"
usa_posting_list, usa_phrase_query = phrase_query(ir, text, noprint=False)

Old Glory
Under the Sun
Homecoming
The Werewolf of Washington
The Man Within
The Legend of Zorro
Skokie
Best of the Best
Out of Control
Raggedy Ann & Andy: A Musical Adventure
G.I. Joe: The Rise of Cobra
Abraham Lincoln: Vampire Hunter
The Phantom
Captain America
D@bbe
Aadhavan
Rock & Rule
Wagon Heels
Miracle
Mambo Italiano
Maa Inti Premayanam
Birthday Boy
2001: A Space Travesty
Hajji Washington
Unholy
Around the World in 80 Days


In [83]:
check_phrase_query(text, usa_posting_list)

#### With spelling correction

In [84]:
mispelled_gb_posting_list, mispelled_gb_phrase_query = phrase_query(ir, "Greatt Britin", spellingCorrection=True, noprint=False)

greatt not found. Did you mean great?
britin not found. Did you mean britain?

Noises Off...
13 Rue Madeleine
The Fox of Glenarvon
International Velvet
Twelve O'Clock High
Saraband for Dead Lovers
Veronico Cruz
Legally Blondes
The Entertainer
The Adams Chronicles
Zulu Dawn
The Three Musketeers
The Story of Adele H.
A Dangerous Man: Lawrence After Arabia
Vigil in the Night
28 Weeks Later
Britannic
The First of the Few
Suez
Hopscotch
It Happened Here
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
The Omen
Patton
Freddie as F.R.O.7
Tristan & Isolde
Garfield: A Tail of Two Kitties
Burn!
Battle of Britain
Tea With Mussolini
The Scarlet Pimpernel
Valiant
The Patriot
Land and Freedom
Sylvia
The Monster X Strikes Back/Attack the G8 Summit
28 Days Later
Jackboots on Whitehall
Omen III: The Final Conflict
Chariots of Fire
John Adams
Souls at Sea
Banned From Television
Satellite in the Sky
The Man Who Shot Liberty Valance
Doomsday
Wild Wild West
633 Squadron
Incense for the Damned

In [85]:
assert gb_phrase_query == mispelled_gb_phrase_query

#### With $k$ steps

In [86]:
text = "United /2 America"
ua_posting_list, ua_phrase_query = phrase_query_ksteps(ir, text, noprint=False)

Old Glory
Under the Sun
Homecoming
The Werewolf of Washington
The Man Within
The Legend of Zorro
Skokie
Best of the Best
Out of Control
Raggedy Ann & Andy: A Musical Adventure
G.I. Joe: The Rise of Cobra
Abraham Lincoln: Vampire Hunter
The Phantom
Captain America
D@bbe
Aadhavan
Rock & Rule
Wagon Heels
Miracle
Mambo Italiano
Maa Inti Premayanam
Birthday Boy
2001: A Space Travesty
Hajji Washington
Unholy
Around the World in 80 Days


Since fortunately there are nor "United America" nor "United States America" in the corpus, we can check that the result of this query is equal to the one of "United States of America".

We have to use the function `ascii` ([documentation](https://docs.python.org/3/library/functions.html#ascii)) because `==` finds them different.

In [87]:
ascii(ua_posting_list) == ascii(usa_posting_list)

True

In [88]:
text = "New /1 city"
nc_posting_list, nc_phrase_query = phrase_query_ksteps(ir, text, noprint=False)

The Lemon Drop Kid
Maria’s Lovers
Anything Else
Wait Until Dark
Munich
The Notorious Elinor Lee
Noises Off...
St. Benny the Dip
Fine Things
London Paris New York
Some Kind of a Nut
The Voyager
Coney Island
Hurricane Streets
Dirty Pretty Things
Bad Lieutenant
Black Swan
Here Comes Mr. Jordan
Nobody Lives Forever
Gangster Exchange
City of Hope
The Women
Vanilla Sky
Birth of the Beatles
Lookin' to Get Out
The City
Saboteur
Pollock
The Jazz Singer
Rebecca of Sunnybrook Farm
Brewster's Millions
The Hudsucker Proxy
The Knickerbocker Buckaroo
The Gumball Rally
June Bride
Sunny Side Up
The Heart Desires
Aviation Vacation
Born in Flames
Pay or Die
The Boys in the Band
The Christmas Tree
Party Wire
The Bonfire of the Vanities
Legal Eagles
The Witches of Oz
Klute
The Taking of Pelham One Two Three
I'll Take Manhattan
Scarface
Change of Heart
Stanley and Livingstone
Short Circuit 2
Outrageous!
Boys' Night Out
Election
25th Hour
The Seven-Ups
Splash, Too
The Way We Live Now
Dirty Laundry
Dream Hous

In this case we find some results also for "New city" so we cannot compare it with the result of the query "New York city".

### Wildcard queries

In [89]:
def check_wildcards_queries(wildcard):
    i = 0
    while not starts_with(ir._index._dictionary[i].term, wildcard):
        passeng = i
        i += 1
    plist = []
    while starts_with(ir._index._dictionary[i].term, wildcard):
        print(ir._index._dictionary[i].term)
        plist.append(ir._index._dictionary[i].posting_list)
        i += 1

    plist = reduce(lambda x, y: x.union(y), plist)
    answer = ir.get_from_corpus(plist)
    return answer

In [90]:
abandon_wildcard_query = trailing_wildcard(ir, "abandon*", noprint=False)

The Lemon Drop Kid
Dark Water
Bhagwan Dada
Killjoy
The Incredibly True Adventure of Two Girls in Love
My Brother's Keeper
Deepstar Six
Indochine
Take A Hard Ride
Northern Lights
Fantastic Planet
Carib Gold
Saved by the Bell: Wedding in Las Vegas
Thanks maa
Farewell My Concubine
St. Benny the Dip
Baby Butch
Daisy Town
The Cold Light of Day
A Big Hand for the Little Lady
Crazy on the Outside
Envy
La Résidence Ylang Ylang
No Good Deed
Come and Get It
Chakushin Ari 2
The Vanishing
The Piano
Toy Story 2
Susan Slept Here
Home Town Story
Peddarayudu
Island
A View to a Kill
Deadly Impact
Mad Love
Freeway Killer
Pudhu Pudhu Arthangal
The Campaign
The Big Chill
The Captive Heart
Blue Gender
Rampage
Kamen Rider World
Together
Himmatwala
Felix the Cat: The Movie
Animal Farm
Maniac Cop 2
Family Guy Presents: Stewie Griffin: The Untold Story
Knocked Up
Paar Magale Paar
The Promise
Saboteur
Pollock
Ten Little Indians
The Mystery of the Marie Celeste
The Seventh Seal
O Lucky Man!
Into the Wild
Chaos
T

In [91]:
abandon_answer = check_wildcards_queries("abandon")

assert abandon_wildcard_query == abandon_answer

abandon
abandoned
abandonedand
abandonedla
abandoning
abandonment
abandonmentcite
abandonmentpart
abandons


In [92]:
passeng_wildcard_query = trailing_wildcard(ir, "passeng*", noprint=False)

L: Change the World
The Voyager
Bordertown
Stowaway
A Ticket to Tomahawk
Runaway Train
Pandemic
Weenie Roast
The Railway Children
A Pleasant Journey
Time Masters
Parachute Battalion
The Cassandra Crossing
Mr. Nobody
Frederick Douglass and the White Negro
Morning Glory
Figures in a Landscape
Aviation Vacation
Starship Troopers: Invasion
Get on the Bus
The Quiet Earth
The Taking of Pelham One Two Three
Jaago
The Legion of the Condemned
Speed 2: Cruise Control
Stanley and Livingstone
Horror Express
Murder on a Honeymoon
Airborne
Mr. Mouse Takes a Trip
Keeper of the Flame
Hooper
Bugs Bunny Rides Again
Third Finger, Left Hand
Pyar To Hona Hi Tha
Betty Boop's Museum
Goke, Body Snatcher from Hell
Open Water
Troublesome Night 5
Payanam
I'll Be Home for Christmas
The Narrow Margin
Death Train
Taxi
Tamango
Home of the Brave
Oh Schuks... I'm Gatvol
This Man Must Die
Dead Men Don't Wear Plaid
Ghostbusters II
Oliver & Company
Limit of Love: Umizaru
Plymouth Adventure
Abduction
Passengers
The Silver

In [93]:
passeng_answer = check_wildcards_queries("passeng")

assert passeng_wildcard_query == passeng_answer

passenger
passenger-side
passengers
passengersjewish
passengersmdashdressed
